## 4a

In [1]:
#Start Spark RDD Session
import re
import sys
from pyspark import SparkConf, SparkContext

conf = SparkConf()
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/18 14:08:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/18 14:08:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
#Point 1
#Read CSV file into RDD
groceries = sc.textFile("grocery_data.csv")
groceries.collect()

['citrus fruit,semi-finished bread,margarine,ready soups',
 'tropical fruit,yogurt,coffee',
 'whole milk',
 'pip fruit,yogurt,cream cheese ,meat spreads',
 'other vegetables,whole milk,condensed milk,long life bakery product',
 'whole milk,butter,yogurt,rice,abrasive cleaner',
 'rolls/buns',
 'other vegetables,UHT-milk,rolls/buns,bottled beer,liquor (appetizer)',
 'pot plants',
 'whole milk,cereals',
 'tropical fruit,other vegetables,white bread,bottled water,chocolate',
 'citrus fruit,tropical fruit,whole milk,butter,curd,yogurt,flour,bottled water,dishes',
 'beef',
 'frankfurter,rolls/buns,soda',
 'chicken,tropical fruit',
 'butter,sugar,fruit/vegetable juice,newspapers',
 'fruit/vegetable juice',
 'packaged fruit/vegetables',
 'chocolate',
 'specialty bar',
 'other vegetables',
 'butter milk,pastry',
 'whole milk',
 'tropical fruit,cream cheese ,processed cheese,detergent,newspapers',
 'tropical fruit,root vegetables,other vegetables,frozen dessert,rolls/buns,flour,sweet spreads,sal

In [3]:
#Point 2
#Strip off any trailing spaces and change the name to lower case.
groceries = groceries.map(lambda line: line.strip().lower())
#Show content
groceries.collect()

['citrus fruit,semi-finished bread,margarine,ready soups',
 'tropical fruit,yogurt,coffee',
 'whole milk',
 'pip fruit,yogurt,cream cheese ,meat spreads',
 'other vegetables,whole milk,condensed milk,long life bakery product',
 'whole milk,butter,yogurt,rice,abrasive cleaner',
 'rolls/buns',
 'other vegetables,uht-milk,rolls/buns,bottled beer,liquor (appetizer)',
 'pot plants',
 'whole milk,cereals',
 'tropical fruit,other vegetables,white bread,bottled water,chocolate',
 'citrus fruit,tropical fruit,whole milk,butter,curd,yogurt,flour,bottled water,dishes',
 'beef',
 'frankfurter,rolls/buns,soda',
 'chicken,tropical fruit',
 'butter,sugar,fruit/vegetable juice,newspapers',
 'fruit/vegetable juice',
 'packaged fruit/vegetables',
 'chocolate',
 'specialty bar',
 'other vegetables',
 'butter milk,pastry',
 'whole milk',
 'tropical fruit,cream cheese ,processed cheese,detergent,newspapers',
 'tropical fruit,root vegetables,other vegetables,frozen dessert,rolls/buns,flour,sweet spreads,sal

In [4]:
#Point 2
#Find total number of transactions
groceries.count()

9835

In [5]:
#Point 3
#Find most number of items in basket and show the count and items in basket.

#Split each word in each line of transaction
items = groceries.map(lambda line: line.split(','))
#Count the number of words in each line
no_of_items = items.map(lambda amount: (len(amount), amount))
#Check to see which line has the highest count
max_items = no_of_items.reduce(lambda a, b: a if a[0] >= b[0] else b)

print("Basket Items: ", max_items[0])
print("Number of Items: ", max_items[1])

Basket Items:  32
Number of Items:  ['frankfurter', 'sausage', 'liver loaf', 'ham', 'chicken', 'beef', 'citrus fruit', 'tropical fruit', 'root vegetables', 'other vegetables', 'whole milk', 'butter', 'curd', 'yogurt', 'whipped/sour cream', 'beverages', 'soft cheese', 'hard cheese', 'cream cheese ', 'mayonnaise', 'domestic eggs', 'rolls/buns', 'roll products ', 'flour', 'pasta', 'margarine', 'specialty fat', 'sugar', 'soups', 'skin care', 'hygiene articles', 'candles']


In [6]:
#Point 4
#Find unique number of items.
groceries.flatMap(lambda line: line.split(',')).distinct().count()

171

## 4b

In [7]:
#Find all items in transaction
total_items = groceries.flatMap(lambda line: line.split(','))
#Count number of unique items in all transactions
count_distinct = total_items.countByValue().items()
#Sort the items in descending order
sorted_count = sorted(count_distinct, key=lambda x: x[1], reverse=True)

In [8]:
#Point 1
#Display the top 20 products with count and percentage in a new RDD
display_output_top20 = sc.parallelize([(item, count, round((count / groceries.count()) * 100, 2)) for item, count in sorted_count[:20]])
display_output_top20.collect()

[('whole milk', 2513, 25.55),
 ('other vegetables', 1903, 19.35),
 ('rolls/buns', 1809, 18.39),
 ('soda', 1715, 17.44),
 ('yogurt', 1372, 13.95),
 ('bottled water', 1087, 11.05),
 ('root vegetables', 1072, 10.9),
 ('tropical fruit', 1032, 10.49),
 ('shopping bags', 969, 9.85),
 ('sausage', 924, 9.4),
 ('pastry', 875, 8.9),
 ('citrus fruit', 814, 8.28),
 ('bottled beer', 792, 8.05),
 ('newspapers', 785, 7.98),
 ('canned beer', 764, 7.77),
 ('pip fruit', 744, 7.56),
 ('fruit/vegetable juice', 711, 7.23),
 ('whipped/sour cream', 705, 7.17),
 ('brown bread', 638, 6.49),
 ('domestic eggs', 624, 6.34)]

In [9]:
#Point 2
#Display the bottom 20 products with count and percentage in a new RDD
display_output_bottom20 = sc.parallelize([(item, count, round((count / groceries.count()) * 100, 2)) for item, count in sorted_count[-20:]])
display_output_bottom20.collect()

[('organic products', 16, 0.16),
 ('honey', 15, 0.15),
 ('roll products', 15, 0.15),
 ('decalcifier', 15, 0.15),
 ('cream', 13, 0.13),
 ('frozen fruits', 12, 0.12),
 ('hair spray', 11, 0.11),
 ('rubbing alcohol', 10, 0.1),
 ('liqueur', 9, 0.09),
 ('make up remover', 8, 0.08),
 ('salad dressing', 8, 0.08),
 ('whisky', 8, 0.08),
 ('toilet cleaner', 7, 0.07),
 ('baby cosmetics', 6, 0.06),
 ('frozen chicken', 6, 0.06),
 ('bags', 4, 0.04),
 ('kitchen utensil', 4, 0.04),
 ('preservation products', 2, 0.02),
 ('baby food', 1, 0.01),
 ('sound storage medium', 1, 0.01)]

## 4c

In [10]:
#Point 1
#Assign index to each transaction in RDD
comb_groceries = groceries.zipWithIndex()
comb_groceries.collect()

[('citrus fruit,semi-finished bread,margarine,ready soups', 0),
 ('tropical fruit,yogurt,coffee', 1),
 ('whole milk', 2),
 ('pip fruit,yogurt,cream cheese ,meat spreads', 3),
 ('other vegetables,whole milk,condensed milk,long life bakery product', 4),
 ('whole milk,butter,yogurt,rice,abrasive cleaner', 5),
 ('rolls/buns', 6),
 ('other vegetables,uht-milk,rolls/buns,bottled beer,liquor (appetizer)', 7),
 ('pot plants', 8),
 ('whole milk,cereals', 9),
 ('tropical fruit,other vegetables,white bread,bottled water,chocolate', 10),
 ('citrus fruit,tropical fruit,whole milk,butter,curd,yogurt,flour,bottled water,dishes',
  11),
 ('beef', 12),
 ('frankfurter,rolls/buns,soda', 13),
 ('chicken,tropical fruit', 14),
 ('butter,sugar,fruit/vegetable juice,newspapers', 15),
 ('fruit/vegetable juice', 16),
 ('packaged fruit/vegetables', 17),
 ('chocolate', 18),
 ('specialty bar', 19),
 ('other vegetables', 20),
 ('butter milk,pastry', 21),
 ('whole milk', 22),
 ('tropical fruit,cream cheese ,processe

In [11]:
#Point 2
#Find all possible combinations of 2 items

import itertools
#Create a function to determine the combinations. The function takes in the RDD
def item_combo(transaction):
    #It splits the items and index and splits the items seperated by a comma
    items, index = transaction
    items = items.split(',')
    #Create empty list to return the combinations
    combinations = []
    #Only take transactions that have more than 2 items, to count the combinations
    if len(items) >= 2:
        #use itertools.combinations to perform the combination of 2 items
        for combo in itertools.combinations(items, 2):
            #only append combinations of 2 items to the list.
            if len(combo) >= 2:
                combinations.append((combo, index))
        return combinations
    else:
        return []

#apply the function to RDD
sorted_combo = comb_groceries.flatMap(item_combo)
#Filter out empty lists
final_sort = sorted_combo.filter(lambda x: x)
#sort by first letter of first item
final_sort_alphabetically = final_sort.sortByKey(lambda x: x[0][0])
final_sort_alphabetically.collect()

[(('abrasive cleaner', 'bathroom cleaner'), 8930),
 (('abrasive cleaner', 'candles'), 3026),
 (('abrasive cleaner', 'cleaner'), 308),
 (('abrasive cleaner', 'cleaner'), 1228),
 (('abrasive cleaner', 'cleaner'), 5966),
 (('abrasive cleaner', 'cleaner'), 6927),
 (('abrasive cleaner', 'cleaner'), 7359),
 (('abrasive cleaner', 'cling film/bags'), 3026),
 (('abrasive cleaner', 'cling film/bags'), 6681),
 (('abrasive cleaner', 'dental care'), 6169),
 (('abrasive cleaner', 'dishes'), 3026),
 (('abrasive cleaner', 'dishes'), 7359),
 (('abrasive cleaner', 'dishes'), 9766),
 (('abrasive cleaner', 'flower (seeds)'), 741),
 (('abrasive cleaner', 'house keeping products'), 5966),
 (('abrasive cleaner', 'house keeping products'), 9790),
 (('abrasive cleaner', 'hygiene articles'), 1147),
 (('abrasive cleaner', 'hygiene articles'), 1228),
 (('abrasive cleaner', 'hygiene articles'), 5610),
 (('abrasive cleaner', 'hygiene articles'), 8173),
 (('abrasive cleaner', 'light bulbs'), 1340),
 (('abrasive clea

In [12]:
#Point 2
#Count of combination records
final_sort_alphabetically.count()

137278

In [13]:
#Point 3
#Find list of transaction indices that are for each item pair
final_final_sort = final_sort_alphabetically.groupByKey().mapValues(list)
final_final_sort.collect()

[(('abrasive cleaner', 'bathroom cleaner'), [8930]),
 (('abrasive cleaner', 'hygiene articles'), [1147, 1228, 5610, 8173]),
 (('abrasive cleaner', 'newspapers'), [741, 3148, 6169, 6320, 6753]),
 (('abrasive cleaner', 'shopping bags'), [637, 1228, 6169]),
 (('abrasive cleaner', 'soap'), [6753]),
 (('artif. sweetener', 'cake bar'), [5033]),
 (('artif. sweetener', 'candles'), [4190, 5891, 9341]),
 (('artif. sweetener', 'canned fruit'), [4172]),
 (('artif. sweetener', 'cat food'), [9691]),
 (('artif. sweetener', 'cereals'), [4190]),
 (('artif. sweetener', 'chocolate marshmallow'), [999]),
 (('artif. sweetener', 'cleaner'), [7097]),
 (('artif. sweetener', 'cling film/bags'), [2584]),
 (('artif. sweetener', 'coffee'), [1976, 2584, 8145, 9691]),
 (('artif. sweetener', 'dental care'), [692]),
 (('artif. sweetener', 'detergent'), [4587, 9211]),
 (('artif. sweetener', 'dish cleaner'), [2337, 6986]),
 (('artif. sweetener', 'dishes'), [2337]),
 (('artif. sweetener', 'dog food'), [1976]),
 (('artif

In [14]:
#Point 3
final_final_sort = final_sort_alphabetically.groupByKey().mapValues(list)
final_final_sort.count()

9677

In [15]:
#Point 4
#find number of times each pair occurs and sort from highest to lowest counts
total_times = final_final_sort.map(lambda x: (x[0], len(x[1]))).sortBy(lambda x: x[1], ascending=False)
total_times.collect()

[(('other vegetables', 'whole milk'), 736),
 (('whole milk', 'rolls/buns'), 557),
 (('whole milk', 'yogurt'), 551),
 (('root vegetables', 'whole milk'), 481),
 (('root vegetables', 'other vegetables'), 466),
 (('other vegetables', 'yogurt'), 427),
 (('other vegetables', 'rolls/buns'), 419),
 (('tropical fruit', 'whole milk'), 416),
 (('whole milk', 'soda'), 394),
 (('rolls/buns', 'soda'), 377),
 (('tropical fruit', 'other vegetables'), 353),
 (('yogurt', 'rolls/buns'), 338),
 (('whole milk', 'bottled water'), 338),
 (('whole milk', 'pastry'), 327),
 (('other vegetables', 'soda'), 322),
 (('whole milk', 'whipped/sour cream'), 317),
 (('sausage', 'rolls/buns'), 301),
 (('citrus fruit', 'whole milk'), 300),
 (('pip fruit', 'whole milk'), 296),
 (('whole milk', 'domestic eggs'), 295),
 (('sausage', 'whole milk'), 294),
 (('tropical fruit', 'yogurt'), 288),
 (('bottled water', 'soda'), 285),
 (('other vegetables', 'whipped/sour cream'), 284),
 (('citrus fruit', 'other vegetables'), 284),
 (

In [16]:
total_times.count()

9677

## 4d

In [17]:
#Point 1
#Calculate support metric

#my assumption of N in support metric which is the total number of transactions 
total_transactions = groceries.count()

support = total_times.map(lambda x: ((x[0][0], x[0][1]), (x[1], round(x[1] / total_transactions * 100,2))))
support.collect()

[(('other vegetables', 'whole milk'), (736, 7.48)),
 (('whole milk', 'rolls/buns'), (557, 5.66)),
 (('whole milk', 'yogurt'), (551, 5.6)),
 (('root vegetables', 'whole milk'), (481, 4.89)),
 (('root vegetables', 'other vegetables'), (466, 4.74)),
 (('other vegetables', 'yogurt'), (427, 4.34)),
 (('other vegetables', 'rolls/buns'), (419, 4.26)),
 (('tropical fruit', 'whole milk'), (416, 4.23)),
 (('whole milk', 'soda'), (394, 4.01)),
 (('rolls/buns', 'soda'), (377, 3.83)),
 (('tropical fruit', 'other vegetables'), (353, 3.59)),
 (('yogurt', 'rolls/buns'), (338, 3.44)),
 (('whole milk', 'bottled water'), (338, 3.44)),
 (('whole milk', 'pastry'), (327, 3.32)),
 (('other vegetables', 'soda'), (322, 3.27)),
 (('whole milk', 'whipped/sour cream'), (317, 3.22)),
 (('sausage', 'rolls/buns'), (301, 3.06)),
 (('citrus fruit', 'whole milk'), (300, 3.05)),
 (('pip fruit', 'whole milk'), (296, 3.01)),
 (('whole milk', 'domestic eggs'), (295, 3.0)),
 (('sausage', 'whole milk'), (294, 2.99)),
 (('tro

In [18]:
support.count()

9677

In [19]:
support.take(20)

[(('other vegetables', 'whole milk'), (736, 7.48)),
 (('whole milk', 'rolls/buns'), (557, 5.66)),
 (('whole milk', 'yogurt'), (551, 5.6)),
 (('root vegetables', 'whole milk'), (481, 4.89)),
 (('root vegetables', 'other vegetables'), (466, 4.74)),
 (('other vegetables', 'yogurt'), (427, 4.34)),
 (('other vegetables', 'rolls/buns'), (419, 4.26)),
 (('tropical fruit', 'whole milk'), (416, 4.23)),
 (('whole milk', 'soda'), (394, 4.01)),
 (('rolls/buns', 'soda'), (377, 3.83)),
 (('tropical fruit', 'other vegetables'), (353, 3.59)),
 (('yogurt', 'rolls/buns'), (338, 3.44)),
 (('whole milk', 'bottled water'), (338, 3.44)),
 (('whole milk', 'pastry'), (327, 3.32)),
 (('other vegetables', 'soda'), (322, 3.27)),
 (('whole milk', 'whipped/sour cream'), (317, 3.22)),
 (('sausage', 'rolls/buns'), (301, 3.06)),
 (('citrus fruit', 'whole milk'), (300, 3.05)),
 (('pip fruit', 'whole milk'), (296, 3.01)),
 (('whole milk', 'domestic eggs'), (295, 3.0))]

In [20]:
total_times_bottom20 = final_final_sort.map(lambda x: (x[0], len(x[1]))).sortBy(lambda x: x[1], ascending=True)
support_bottom20 = total_times_bottom20.map(lambda x: ((x[0][0], x[0][1]), (x[1], round(x[1] / total_transactions * 100,2))))
support_bottom20.take(20)

[(('abrasive cleaner', 'bathroom cleaner'), (1, 0.01)),
 (('abrasive cleaner', 'soap'), (1, 0.01)),
 (('artif. sweetener', 'cake bar'), (1, 0.01)),
 (('artif. sweetener', 'canned fruit'), (1, 0.01)),
 (('artif. sweetener', 'cat food'), (1, 0.01)),
 (('artif. sweetener', 'cereals'), (1, 0.01)),
 (('artif. sweetener', 'chocolate marshmallow'), (1, 0.01)),
 (('artif. sweetener', 'cleaner'), (1, 0.01)),
 (('artif. sweetener', 'cling film/bags'), (1, 0.01)),
 (('artif. sweetener', 'dental care'), (1, 0.01)),
 (('artif. sweetener', 'dishes'), (1, 0.01)),
 (('artif. sweetener', 'dog food'), (1, 0.01)),
 (('artif. sweetener', 'house keeping products'), (1, 0.01)),
 (('artif. sweetener', 'male cosmetics'), (1, 0.01)),
 (('artif. sweetener', 'organic products'), (1, 0.01)),
 (('artif. sweetener', 'pot plants'), (1, 0.01)),
 (('artif. sweetener', 'ready soups'), (1, 0.01)),
 (('artif. sweetener', 'red/blush wine'), (1, 0.01)),
 (('artif. sweetener', 'salty snack'), (1, 0.01)),
 (('artif. sweetene

## 4e

In [21]:
#Point 1
#Assign index to groceries
perm_groceries = groceries.zipWithIndex()
perm_groceries.collect()

[('citrus fruit,semi-finished bread,margarine,ready soups', 0),
 ('tropical fruit,yogurt,coffee', 1),
 ('whole milk', 2),
 ('pip fruit,yogurt,cream cheese ,meat spreads', 3),
 ('other vegetables,whole milk,condensed milk,long life bakery product', 4),
 ('whole milk,butter,yogurt,rice,abrasive cleaner', 5),
 ('rolls/buns', 6),
 ('other vegetables,uht-milk,rolls/buns,bottled beer,liquor (appetizer)', 7),
 ('pot plants', 8),
 ('whole milk,cereals', 9),
 ('tropical fruit,other vegetables,white bread,bottled water,chocolate', 10),
 ('citrus fruit,tropical fruit,whole milk,butter,curd,yogurt,flour,bottled water,dishes',
  11),
 ('beef', 12),
 ('frankfurter,rolls/buns,soda', 13),
 ('chicken,tropical fruit', 14),
 ('butter,sugar,fruit/vegetable juice,newspapers', 15),
 ('fruit/vegetable juice', 16),
 ('packaged fruit/vegetables', 17),
 ('chocolate', 18),
 ('specialty bar', 19),
 ('other vegetables', 20),
 ('butter milk,pastry', 21),
 ('whole milk', 22),
 ('tropical fruit,cream cheese ,processe

In [22]:
#Point 2
#Find all possible permutations

import itertools

#create a function to identify all possible permutations. I have reused the same function from finding
#combinations.
def item_perm(perm_transaction):
    perm_items, perm_index = perm_transaction
    perm_items = perm_items.split(',')
    permutations = []
    if len(perm_items) >= 2:
        for perm in itertools.permutations(perm_items, 2):
            if len(perm) >= 2:
                permutations.append((perm, perm_index))
        return permutations
    else:
        return []
    
sorted_perm = perm_groceries.flatMap(item_perm)
final_perm_sort = sorted_perm.filter(lambda x: x)
final_perm_sort_alphabetically = final_perm_sort.sortByKey(lambda x: x[0][0])
final_perm_sort_alphabetically.collect()

[(('abrasive cleaner', 'artif. sweetener'), 33),
 (('abrasive cleaner', 'bathroom cleaner'), 8930),
 (('abrasive cleaner', 'beef'), 294),
 (('abrasive cleaner', 'beef'), 637),
 (('abrasive cleaner', 'beef'), 1147),
 (('abrasive cleaner', 'beef'), 1228),
 (('abrasive cleaner', 'beef'), 3148),
 (('abrasive cleaner', 'beef'), 6753),
 (('abrasive cleaner', 'berries'), 33),
 (('abrasive cleaner', 'berries'), 294),
 (('abrasive cleaner', 'berries'), 867),
 (('abrasive cleaner', 'berries'), 1228),
 (('abrasive cleaner', 'berries'), 6904),
 (('abrasive cleaner', 'berries'), 8930),
 (('abrasive cleaner', 'beverages'), 867),
 (('abrasive cleaner', 'bottled beer'), 1340),
 (('abrasive cleaner', 'bottled beer'), 5610),
 (('abrasive cleaner', 'bottled beer'), 5809),
 (('abrasive cleaner', 'bottled beer'), 6753),
 (('abrasive cleaner', 'bottled beer'), 6927),
 (('abrasive cleaner', 'bottled beer'), 8930),
 (('abrasive cleaner', 'bottled water'), 1147),
 (('abrasive cleaner', 'bottled water'), 6681),

In [23]:
#Point 2
#Count number of permutations
final_perm_sort_alphabetically.count()

274556

In [24]:
#Point 2
#Find list of transaction indices
final_final_perm_sort = final_perm_sort_alphabetically.groupByKey().mapValues(list)
final_final_perm_sort.collect()

[(('abrasive cleaner', 'bathroom cleaner'), [8930]),
 (('abrasive cleaner', 'bottled beer'), [1340, 5610, 5809, 6753, 6927, 8930]),
 (('abrasive cleaner', 'bottled water'), [1147, 6681, 8280]),
 (('abrasive cleaner', 'butter milk'), [6320, 8280]),
 (('abrasive cleaner', 'candy'), [308, 3026, 8173]),
 (('abrasive cleaner', 'canned fish'), [5610, 6927]),
 (('abrasive cleaner', 'canned vegetables'), [1147, 5610]),
 (('abrasive cleaner', 'cream cheese '), [1228, 5610, 7542, 8173]),
 (('abrasive cleaner', 'frozen dessert'), [7542]),
 (('abrasive cleaner', 'grapes'), [8173]),
 (('abrasive cleaner', 'hamburger meat'), [6740, 9766]),
 (('abrasive cleaner', 'herbs'), [1147, 6740]),
 (('abrasive cleaner', 'hygiene articles'), [1147, 1228, 5610, 8173]),
 (('abrasive cleaner', 'instant coffee'), [5610]),
 (('abrasive cleaner', 'jam'), [5809]),
 (('abrasive cleaner', 'ketchup'), [5610]),
 (('abrasive cleaner', 'long life bakery product'), [854, 6904, 8173]),
 (('abrasive cleaner', 'margarine'), [85

In [25]:
#Point 2
#Count list of indices
final_final_perm_sort.count()

19354

In [26]:
#Point 3
#find number of times each pair occurs and sort from highest to lowest counts

total_times_perm = final_final_perm_sort.map(lambda x: (x[0], len(x[1]))).sortBy(lambda x: x[1], ascending=False)
total_times_perm.collect()

[(('other vegetables', 'whole milk'), 736),
 (('whole milk', 'other vegetables'), 736),
 (('rolls/buns', 'whole milk'), 557),
 (('whole milk', 'rolls/buns'), 557),
 (('whole milk', 'yogurt'), 551),
 (('yogurt', 'whole milk'), 551),
 (('root vegetables', 'whole milk'), 481),
 (('whole milk', 'root vegetables'), 481),
 (('other vegetables', 'root vegetables'), 466),
 (('root vegetables', 'other vegetables'), 466),
 (('other vegetables', 'yogurt'), 427),
 (('yogurt', 'other vegetables'), 427),
 (('other vegetables', 'rolls/buns'), 419),
 (('rolls/buns', 'other vegetables'), 419),
 (('tropical fruit', 'whole milk'), 416),
 (('whole milk', 'tropical fruit'), 416),
 (('soda', 'whole milk'), 394),
 (('whole milk', 'soda'), 394),
 (('rolls/buns', 'soda'), 377),
 (('soda', 'rolls/buns'), 377),
 (('other vegetables', 'tropical fruit'), 353),
 (('tropical fruit', 'other vegetables'), 353),
 (('rolls/buns', 'yogurt'), 338),
 (('yogurt', 'rolls/buns'), 338),
 (('bottled water', 'whole milk'), 338),

In [27]:
total_times_perm.count()

19354

In [28]:
#Point 4
#Compute Frexuency (X) and sort count from highest to lowest
total_X = final_final_perm_sort.flatMap(lambda x: x[0]).groupBy(lambda x: x).map(lambda x: (x[0], len(x[1]))).sortBy(lambda x: x[1], ascending=False)
total_X.collect()

[('whole milk', 336),
 ('other vegetables', 334),
 ('rolls/buns', 332),
 ('yogurt', 332),
 ('tropical fruit', 330),
 ('soda', 328),
 ('butter', 326),
 ('pip fruit', 324),
 ('root vegetables', 322),
 ('shopping bags', 322),
 ('citrus fruit', 320),
 ('pastry', 320),
 ('whipped/sour cream', 320),
 ('white bread', 318),
 ('curd', 316),
 ('fruit/vegetable juice', 316),
 ('sausage', 316),
 ('domestic eggs', 314),
 ('pork', 314),
 ('beef', 312),
 ('frozen vegetables', 312),
 ('napkins', 312),
 ('bottled water', 312),
 ('coffee', 310),
 ('waffles', 310),
 ('chocolate', 310),
 ('frankfurter', 308),
 ('margarine', 308),
 ('newspapers', 308),
 ('dessert', 304),
 ('hamburger meat', 304),
 ('hygiene articles', 304),
 ('bottled beer', 302),
 ('candy', 302),
 ('cream cheese ', 302),
 ('long life bakery product', 302),
 ('specialty chocolate', 302),
 ('berries', 300),
 ('brown bread', 300),
 ('salty snack', 300),
 ('sugar', 298),
 ('grapes', 298),
 ('onions', 298),
 ('hard cheese', 296),
 ('oil', 294)

## 4f

In [29]:
#Point 1
#Calculate confidence metric.
#Join the RDD containing [(X,Y, (pair count)] with [(X),(X count)] and form a new RDD with 
#[(X,Y),(XY pair count, XCount, confidence metric)]

confidence = total_times_perm.map(lambda x: (x[0][0], (x[0][1], x[1]))).join(total_X) \
            .map(lambda x: ((x[1][0][0], x[0]), (x[1][0][1], x[1][1], round((x[1][0][1] / x[1][1]) * 100, 2))))

confidence.collect()

[(('other vegetables', 'whole milk'), (736, 336, 219.05)),
 (('rolls/buns', 'whole milk'), (557, 336, 165.77)),
 (('yogurt', 'whole milk'), (551, 336, 163.99)),
 (('root vegetables', 'whole milk'), (481, 336, 143.15)),
 (('tropical fruit', 'whole milk'), (416, 336, 123.81)),
 (('soda', 'whole milk'), (394, 336, 117.26)),
 (('bottled water', 'whole milk'), (338, 336, 100.6)),
 (('pastry', 'whole milk'), (327, 336, 97.32)),
 (('whipped/sour cream', 'whole milk'), (317, 336, 94.35)),
 (('citrus fruit', 'whole milk'), (300, 336, 89.29)),
 (('pip fruit', 'whole milk'), (296, 336, 88.1)),
 (('domestic eggs', 'whole milk'), (295, 336, 87.8)),
 (('sausage', 'whole milk'), (294, 336, 87.5)),
 (('butter', 'whole milk'), (271, 336, 80.65)),
 (('newspapers', 'whole milk'), (269, 336, 80.06)),
 (('fruit/vegetable juice', 'whole milk'), (262, 336, 77.98)),
 (('curd', 'whole milk'), (257, 336, 76.49)),
 (('brown bread', 'whole milk'), (248, 336, 73.81)),
 (('shopping bags', 'whole milk'), (241, 336, 

In [30]:
#Point 1
#Count number of records
confidence.count()

19354

In [31]:
#Point 2
#Take the top 20 records based on X count
sorted_confidence = confidence.sortBy(lambda x: (x[1][1]), ascending=False)
sorted_confidence.take(20)

[(('other vegetables', 'whole milk'), (736, 336, 219.05)),
 (('rolls/buns', 'whole milk'), (557, 336, 165.77)),
 (('yogurt', 'whole milk'), (551, 336, 163.99)),
 (('root vegetables', 'whole milk'), (481, 336, 143.15)),
 (('tropical fruit', 'whole milk'), (416, 336, 123.81)),
 (('soda', 'whole milk'), (394, 336, 117.26)),
 (('bottled water', 'whole milk'), (338, 336, 100.6)),
 (('pastry', 'whole milk'), (327, 336, 97.32)),
 (('whipped/sour cream', 'whole milk'), (317, 336, 94.35)),
 (('citrus fruit', 'whole milk'), (300, 336, 89.29)),
 (('pip fruit', 'whole milk'), (296, 336, 88.1)),
 (('domestic eggs', 'whole milk'), (295, 336, 87.8)),
 (('sausage', 'whole milk'), (294, 336, 87.5)),
 (('butter', 'whole milk'), (271, 336, 80.65)),
 (('newspapers', 'whole milk'), (269, 336, 80.06)),
 (('fruit/vegetable juice', 'whole milk'), (262, 336, 77.98)),
 (('curd', 'whole milk'), (257, 336, 76.49)),
 (('brown bread', 'whole milk'), (248, 336, 73.81)),
 (('shopping bags', 'whole milk'), (241, 336, 

In [32]:
#Point 3
#Take the bottom 20 records based on X count
sorted_confidence_asc = confidence.sortBy(lambda x: (x[1][1]), ascending=True)
sorted_confidence_asc.take(20)

[(('bottled water', 'sound storage medium'), (1, 18, 5.56)),
 (('candy', 'sound storage medium'), (1, 18, 5.56)),
 (('pastry', 'sound storage medium'), (1, 18, 5.56)),
 (('shopping bags', 'sound storage medium'), (1, 18, 5.56)),
 (('soda', 'sound storage medium'), (1, 18, 5.56)),
 (('cat food', 'sound storage medium'), (1, 18, 5.56)),
 (('frozen potato products', 'sound storage medium'), (1, 18, 5.56)),
 (('ham', 'sound storage medium'), (1, 18, 5.56)),
 (('white bread', 'sound storage medium'), (1, 18, 5.56)),
 (('pastry', 'bags'), (1, 26, 3.85)),
 (('pickled vegetables', 'bags'), (1, 26, 3.85)),
 (('rolls/buns', 'bags'), (1, 26, 3.85)),
 (('tidbits', 'bags'), (1, 26, 3.85)),
 (('yogurt', 'bags'), (1, 26, 3.85)),
 (('frozen potato products', 'bags'), (1, 26, 3.85)),
 (('frozen vegetables', 'bags'), (1, 26, 3.85)),
 (('fruit/vegetable juice', 'bags'), (1, 26, 3.85)),
 (('napkins', 'bags'), (1, 26, 3.85)),
 (('pip fruit', 'bags'), (1, 26, 3.85)),
 (('pork', 'bags'), (1, 26, 3.85))]

In [33]:
sc.stop()